## Connection to a SQL database

In [ ]:
import os
import mysql.connector

# Access environment variable directly (may raise KeyError if not set)
database_name = os.environ['MYSQL_DATABASE']
user = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']

mydb = mysql.connector.connect(
  host="mysql",  # Use service name from docker-compose
  user=user,
  password=password,
  database=database_name
)

mycursor = mydb.cursor()

In [3]:
# Define your table schema
sql = """CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT PRIMARY KEY,
  username VARCHAR(255) NOT NULL,
  email VARCHAR(255) NOT NULL UNIQUE
)"""

mycursor.execute(sql)

In [4]:
sql = "INSERT INTO users (username, email) VALUES (%s, %s)"
val = ("John Doe", "johndoe@example.com")
mycursor.execute(sql, val)

# Insert multiple rows at once using executemany
sql = "INSERT INTO users (username, email) VALUES (%s, %s)"
val = [
  ("Jane Doe", "janedoe@example.com"),
  ("Peter Smith", "petersmith@example.com"),
]
mycursor.executemany(sql, val)

mydb.commit()  # Commit changes to the database

## Conecction to a Virtuoso Triple Store

In [1]:
from rdflib.graph import ConjunctiveGraph as Graph
from rdflib.store import Store
from rdflib.plugin import get as plugin
from rdflib.term import URIRef, Literal

# DSN with connection details
# Virtuoso connection details
username = "dba"
password = "dba"
host = "virtuoso"
port = 8890
dsn = "VOS;UID=%s;PWD=%s;HOST=%s;PORT=%s" % (username, password, host, port) 

# Create Virtuoso store object
Virtuoso = plugin("Virtuoso", Store)
graph = Graph(Virtuoso(dsn))

PluginException: No plugin registered for (Virtuoso, <class 'rdflib.store.Store'>)

In [ ]:
# Define resources
subject = URIRef("http://example.com/resource1")
predicate = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")
object = URIRef("http://example.com/ClassA")

# Add triple to the graph
graph.add((subject, predicate, object))

# Commit changes to Virtuoso
graph.commit()

## Connection to a Anzograph DB

### Aproach 1

In [10]:
import rdflib.plugins.stores.sparqlstore as sprql_store
from rdflib import Namespace ,Graph, URIRef, Literal, BNode, store
from rdflib.namespace import FOAF, RDF
from rdflib.plugins.stores import sparqlstore
from rdflib.graph import DATASET_DEFAULT_GRAPH_ID as default

# store = sprql_store.SPARQLStore("http://anzograph:7070/sparql")
store = sparqlstore.SPARQLUpdateStore()
query_endpoint = 'http://anzograph:7070/sparql'
update_endpoint = 'http://anzograph:7070/sparql'
store.open((query_endpoint, update_endpoint))

# Define example namespace.
ex = Namespace("http://example.org/")

# Define a node.
node = (ex.me, RDF.type, FOAF.Person)

# Open a graph in the open store and set identifier to default graph ID.
graph = Graph(store, identifier=default)

# Add node to graph.
graph.add(node)
graph.

<Graph identifier=urn:x-rdflib:default (<class 'rdflib.graph.Graph'>)>

In [8]:
graph.query(""" SELECT ?g (COUNT(*) as ?count)
WHERE {
	graph ?g{
		?s ?p ?o
	}
}
GROUP BY ?g
ORDER BY DESC(?count)""")

ValueError: You did something wrong formulating either the URI or your SPARQL query

### Aproach 2

In [5]:
import rdflib
import rdflib.plugins.stores.sparqlstore as sprql_store
from rdflib import Graph, URIRef, Literal, BNode, store
from rdflib.namespace import FOAF, RDF

store = sprql_store.SPARQLStore("http://anzograph:7070/sparql")
g = Graph(store=store)

g.query(""" SELECT ?g (COUNT(*) as ?count)
WHERE {
	graph ?g{
		?s ?p ?o
	}
}
GROUP BY ?g
ORDER BY DESC(?count)""")

In [16]:
graph.query("""INSERT DATA {
  <http://example.com/book1> <http://purl.org/dc/elements/1.1/title> "Introduction to SPARQL" .
}""")

AttributeError: 'NoneType' object has no attribute 'split'

### Aproach 3

In [27]:
import azg3

result = azg3.create_dataframe("anzograph:7070" , """ SELECT ?g (COUNT(*) as ?count)
WHERE {
	graph ?g{
		?s ?p ?o
	}
}
GROUP BY ?g
ORDER BY DESC(?count)""")

print(result)

                             g  count
0  <http://example.com/graph1>      3


In [26]:
result = azg3.run_query("anzograph:7070" , """ 
PREFIX my: <http://example.com/my#>  # Replace with your actual prefix

INSERT DATA {
  GRAPH <http://example.com/graph1> {  # Replace with the desired graph IRI
    my:NewObject1 my:newProperty "New Value 1" .
    my:NewObject2 my:anotherProperty "Another Value 2" .
  }
};
""")

print (result)

b''


In [22]:
result = azg3.run_query("anzograph:7070" , """ 
INSERT DATA {
  <http://example.com/book1> <http://purl.org/dc/elements/1.1/title> "Introduction to SPARQL" .
}
"""                          
)
print(result)

b''


In [23]:
result = azg3.create_dataframe("anzograph:7070" , """ ASK {
  <http://example.com/book1> <http://purl.org/dc/elements/1.1/title> "Introduction to SPARQL" .
}""")

print(result)


Empty DataFrame
Columns: []
Index: []
